This notebook will be used to test and demo Barycorrpy: Precise Barycentric Correction for Stellar and Solar Radial Velocities and Time-stamps. The notebook is by the Barrycorrpy team (Shubham Kanodia, Jason Wright, Joe Ninan) and has been edited by Carlos E. Cruz-Arce based on https://github.com/shbhuk/barycorrpy .The orginal notebook can be found here: https://github.com/shbhuk/barycorrpy/wiki/02.-Getting-Started

In [4]:
#How to install
#pip install barycorrpy
#import barycorrpy
#print (barycorrpy.__version__) 

# version 0.3.4 is installed which includes:
## barycentric corrections for Solar observations,
### as well as reflected light observations from other Solar system 
#### objects such as Vesta, Ceres, etc.

Barycorrpy can be used to calculate the barycentric velocity correction for a star with an accuracy of < 1 cm/s . To do this, it takes the following into consideration:

1. Revolution of the Earth to consider position and velocity of the geocenter with respect to the Solar System barycenter
2. Rotation of the Earth
3. Precession, nutation and polar motion of the Earth, along with the above to calculate the position and velocity of the observatory with respect to the geocenter
4. Gravitational time dilation due to objects of the Solar System
5. Leap second offset
6. Proper motion and systemic radial velocity of the star
7. Parallax
8. Shapiro delay (a special case of gravitational time dilation)

We will work off the sample script that comes with the package.
Before we start double check that you have all dependencies installed:
1. astropy version >= 3.2.3
2. numpy version >=1.17.2
3. scipy
4. jplephem
5. astroquery version >= 0.3.10

pip installing barycorrpy should automatically update/install all of these packages but it's always good to be sure.

What we will do is show the various ways barycorrpy can obtain the barycentric corrected RV

In [5]:
#Import all the modules you'll need
from __future__ import print_function
from __future__ import division
from astropy.time import Time
import numpy as np
from barycorrpy import get_BC_vel , exposure_meter_BC_vel , utc_tdb 

In [6]:
#Create a function to run a sample
def run_sample():
        a=[]
        b=0

        JDUTC = 2458000 # Also accepts float input for JDUTC. Verify scale and format

        # Observation of Tau Ceti taken from CTIO (Cerro Tololo Inter-American Observatory) on JD 2458000. 
        # Observatory location manually entered. Stellar positional parameters taken from Hipparcos Catalogue
        result = get_BC_vel(JDUTC=JDUTC, hip_id=8102, lat=-30.169283, longi=-70.806789, alt=2241.9, ephemeris='de430', zmeas=0.0)
        ##get_BC_vel? => JDUTC, starname='',hip_id=None,ra=None,dec=None,epoch=None,pmra=None,pmdec=None,px=None,rv=None,obsname='',lat=0.0,longi=0.0,alt=0.0,zmeas=0.0,ephemeris='de430',leap_dir='/opt/anaconda3/lib/python3.8/site-packages/barycorrpy/data',leap_update=True,predictive=False,SolSystemTarget=None,HorizonsID_type='smallbody',


        if np.isclose(a = result[0], b = 15403.9508, atol = 1e-2, rtol = 0): 
            a.append('result')
            b+=1
            # 15403.95083409 is the actual barycentric RV for this given star
            # result[0] is the barycentric RV in m/s that get_BC_vel spits out


        # Observation of Tau Ceti taken from CTIO on JD 2458000. Observatory location taken from Astropy list.
        # Stellar positional parameters taken from Hipparcos Catalogue
        JDUTC = Time(2458000, format='jd', scale='utc')
        result2  = get_BC_vel(JDUTC=JDUTC, hip_id=8102, obsname='CTIO', ephemeris='de430')

        if np.isclose(a = result2[0], b = 15403.9608, atol = 1e-2, rtol = 0):
            a.append('result2')
            b+=1

        # Observation of Tau Ceti taken from CTIO on JD 2458000,2458010,2458020.
        # Observatory and stellar parameters entered by user.
        # Use DE405 ephemeris ,default is de430
    
        ra=26.0213645867     #Right Ascension in degrees 
        dec=-15.9395557246   #Declination in degrees
        obsname=''           #Name of Observatory 
        lat=-30.169283       #Lattitude of Observatory
        longi=-70.806789     #Longitude of Observatory
        alt=2241.9           #Altitude of Observatory
        epoch = 2451545.0    #Epoch
        pmra = -1721.05      #Proper motion in RA [mas/year]. Eg. PMRA = d(RA)/dt * cos(dec). Default is 0. [mas] = milli arcsecond 
        pmdec = 854.16       #Proper motion in Dec [mas/year]. Default is 0. [mas] = milli arcsecond
        px = 273.96          #Parallax of target [mas]. Default is 0.  [mas] = milli arcsecond
        
        rv = 0.0             #Radial Velocity of Target [m/s]. Default is 0. This is the bulk RV (systemic) of the target at the ~100 km/s precision. 
                             ##Can be ignored for most targets but for high velocity stars. 
                             ###This does not include the barycentric velocity and is only required to correct for proper motion and secular acceleration.
        
        zmeas=0.0            #Measured redshift (e.g., the result of cross correlation with template spectrum). Default is 0.
                             ##The redshift measured by the spectrograph before any barycentric correction. 
                             ###Therefore zmeas includes the barycentric velocity of the observatory.
        JDUTC=[2458000,2458000.00001,2458000.00002] # Can also enter JDUTC as float instead of Astropy Time Object
        ephemeris='https://naif.jpl.nasa.gov/pub/naif/generic_kernels/spk/planets/a_old_versions/de405.bsp'

        result3=get_BC_vel(JDUTC=JDUTC, ra=ra, dec=dec, obsname=obsname, lat=lat, longi=longi, alt=alt, pmra=pmra,
            pmdec=pmdec, px=px, rv=rv, zmeas=zmeas,epoch=epoch, ephemeris=ephemeris, leap_update=True)

        if np.allclose([result3[0][0],result3[0][1],result3[0][2]],[15407.4860,15407.4723,15407.4586],atol = 1e-2, rtol = 0):
            a.append('result3')
            b+=1



        # Exposure meter calculation
        "In BCPy, we have now included a function which takes an array with the flux values at each of the JDUTC time steps (flux time stamp from Exposure meter)"
        "and uses that to calculate the weighted barycentric velocity for the observation."
        "Therefore the correction is calculated at each JDUTC and an average weighted by the exposure meter fluxes is returned."
        "The Exposure meter should have a cadence of minimum 1 min for cm/s."
        
        "The difference between result3 and result4 is that in result4 they are not 3 separate observation times, but the various exposure meter time stamps"
        "(i.e. flux time stamps) which are being used to correct for changes in the flux level (as observed by exposure meter) during a single observation" 
        "Further the 4th result contains the JDUTCMID, the flux weighted midpoint of the observation."
        
        "Calculate Barycentric velocity weighted by flux from exposure meter to account for long exposure time."
        "Enter JDUTC and expmeterflux (exposure meter flux) from exposure meter readings to calculate barycentric velocity correction for exposure."
        flux = [4.5,1.5,2] # Same number of elements as JDUTC
                           ##JDUTCMID is the flux weighted midpoint of the observation

        result4,JDUTCMID,warning4,status4=exposure_meter_BC_vel(JDUTC=JDUTC, expmeterflux = flux,
            ra=ra, dec=dec, obsname=obsname, lat=lat, longi=longi, alt=alt, pmra=pmra,
            pmdec=pmdec, px=px, rv=rv, zmeas=zmeas, epoch=epoch, ephemeris=ephemeris, leap_update=True)  #leap_update checks for updated Leaps files. True will use the newest one

        if np.isclose(a = result4, b = 15407.4765, atol = 1e-2, rtol = 0):
            a.append('result4')
            b+=1
            

        # JDUTC to BJDTDB time converter
        corr_time = utc_tdb.JDUTC_to_BJDTDB(JDUTC, hip_id=8102, lat=-30.169283, longi=-70.806789, alt=2241.9)

        if np.allclose([corr_time[0]], [2458000.00505211,  2458000.00506211,  2458000.00507211], atol = 1e-7, rtol = 0):
            a.append('corr_time')
            b+=1

        # Predictive Mode
        result5 = get_BC_vel(JDUTC=2458000, hip_id=8102, lat=-30.169283, longi=-70.806789, alt=2241.9, ephemeris='de430', zmeas=0.0, predictive=True)

        if np.isclose(a = result5[0], b = -15403.15938, atol = 1e-2, rtol = 0):
            a.append('result5')
            b+=1

        result6 = get_BC_vel(JDUTC=2458000, lat=-30.169138888, longi=-70.805888, alt=2379.5, zmeas=0.0, SolSystemTarget='Sun')

        if np.isclose(a=result6[0], b=819.4474, atol=1e-2, rtol=0):
            a.append('result6')
            b+=1


        if b==7:
            print('***********SUCCESS**************\nAll barycentric correction velocities match expected values to 1 cm/s\n')
        else:
            print('{} out of 6 results match. Compare outputs vs those on the github wiki. Check others - \n'.format(b,a))

        return result, result2, result3, result4, JDUTCMID, warning4, status4, corr_time, result5, result6
        

In [8]:
#Now to see the results
(run_sample())

***********SUCCESS**************
All barycentric correction velocities match expected values to 1 cm/s



((array([15403.95083409]),
  [['Warning: Assuming input time is in UTC JD. Pass astropy Time object to avoid this warning.'],
   'Following are the stellar positional parameters being used - ',
   {'ra': 26.021364586713265,
    'dec': -15.939555724635493,
    'pmra': -1721.05,
    'pmdec': 854.16,
    'px': 273.96,
    'epoch': 2448349.0625},
   [],
   []],
  1),
 (array([15403.96081446]),
  ['Warning: Taking observatory coordinates from Astropy Observatory database. Verify precision. Latitude = -30.165278  Longitude = -70.815000  Altitude = 2215.000000',
   'Following are the stellar positional parameters being used - ',
   {'ra': 26.021364586713265,
    'dec': -15.939555724635493,
    'pmra': -1721.05,
    'pmdec': 854.16,
    'px': 273.96,
    'epoch': 2448349.0625},
   [],
   []],
  1),
 (array([15407.48590935, 15407.47225001, 15407.45859206]),
  [['Warning: Assuming input time is in UTC JD. Pass astropy Time object to avoid this warning.'],
   'Following are the stellar positional

Moving on..
You can also choose a star and get all of it's stellar data from SIMBAD.

In [9]:
from barycorrpy.utils import get_stellar_data
get_stellar_data("Proxima B")

({'ra': 217.42893791666663,
  'dec': -62.679491666666664,
  'pmra': -3781.306,
  'pmdec': 769.766,
  'px': 768.5004,
  'rv': None,
  'epoch': 2451545.0},
 ['Proxima B queried from SIMBAD.',
  'Masked values present in queried dataset',
  "Values queried from SIMBAD are {'ra': 217.42893791666663, 'dec': -62.679491666666664, 'pmra': -3781.306, 'pmdec': 769.766, 'px': 768.5004, 'rv': None, 'epoch': 2451545.0}"])

Moving on..Additionally Barycorrpy also comes with a JDUTC to BJDTDB converter

JDUTC to BJDTDB converter

Include the following:

1. Clock Correction - To correct for difference between UTC and TDB time scales.
2. Geometric Correction - Light travel time from observatory to Solar System Barycenter.
3. Einstein Correction - Relativistic correction since Earth is not an inertial frame.

In [10]:
from astropy.time import Time
from barycorrpy import utc_tdb
JDUTC = Time(2458000, format='jd', scale='utc')
utc_tdb.JDUTC_to_BJDTDB(JDUTC,hip_id=8102, lat=-30.169283, longi=-70.806789, alt=2241.9)
#array is the conversion of JD 2458000 to BJD

(array([2458000.00505211]),
 ['Following are the stellar positional parameters being used - ',
  {'ra': 26.021364586713265,
   'dec': -15.939555724635493,
   'pmra': -1721.05,
   'pmdec': 854.16,
   'px': 273.96,
   'rv': 0.0,
   'epoch': 2448349.0625},
  [],
  []],
 1)

For Context JDUTC is Julian Date and Universal Coordinate Time and BJDTBD is Barycentric Julian Dates and Barycentric Dynamical Time")

In [ ]:
exposure_meter_BC_vel?

In [ ]:
batman = get_BC_vel(JDUTC=JDUTC, hip_id=8102, lat=-30.169283, longi=-70.806789, alt=2241.9, ephemeris='de430', zmeas=0.0)
(batman)

